### Huggingface and gradio

A different library for sharing applications called `gradio` is becoming popular with easy deep learning model integrations.  For this assignment, your goal is to write a basic tutorial that:

- Discusses the basics of building a `gradio` app with rows, columns, images, input fields, and sliders.  Demonstrate this with a `gradio` app of your imagination (different than docs creativity matters!).
- Demonstrates how to use the huggingface `transformers` library and its pipelines to build a `gradio` app. [here](https://www.gradio.app/guides/using-hugging-face-integrations)

Create a github repository with your tutorial notebook and a README with a paragraph overview of the tutorial and description of huggingface model and app.

Tutorial Goal
Build a basic Gradio app using a Blocks layout (rows/columns), an image, text input fields, and a slider. Then integrate a Hugging Face `transformers` pipeline into the app so user input is processed by a pretrained model and results are displayed in the UI.

In [2]:
!pip -q install -U gradio transformers accelerate torch
import gradio as gr
from transformers import pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 133.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 588.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28

In [4]:
# Hugging Face transformers pipeline (pretrained sentiment model)
sentiment_pipe = pipeline(
    task="sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=-1  # CPU for Colab stability
)

def analyze_review(review_text: str, confidence_threshold: float):
    """
    Runs a HF sentiment pipeline and returns:
    1) a summary string
    2) a 2-class probability dict
    3) a threshold-based verdict string
    """
    try:
        if not review_text or not review_text.strip():
            return "Please enter text.", {"NEGATIVE": 0.0, "POSITIVE": 0.0}, "No text provided."

        out = sentiment_pipe(review_text[:2000])[0]
        label = str(out["label"]).upper()
        score = float(out["score"])

        if label == "POSITIVE":
            probs = {"NEGATIVE": float(1.0 - score), "POSITIVE": score}
        else:
            probs = {"NEGATIVE": score, "POSITIVE": float(1.0 - score)}

        verdict = (
            f"Meets threshold ({confidence_threshold:.2f})."
            if score >= confidence_threshold
            else f"Below threshold ({confidence_threshold:.2f})."
        )

        summary = f"Prediction: {label} (confidence={score:.3f})"
        return summary, probs, verdict

    except Exception as e:
        return f"ERROR: {type(e).__name__}: {e}", {"NEGATIVE": 0.0, "POSITIVE": 0.0}, "ERROR"


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Gradio Layout (Rows/Columns, Image, Inputs, Slider).
This section builds a Gradio Blocks app using a row/column layout.
The left column contains an image, a confidence-threshold slider, and helper controls.
The right column contains the main text input and model outputs.
Clicking the button runs the Hugging Face pipeline and displays the prediction results.

In [6]:
HEADER_IMAGE_URL = "https://huggingface.co/front/assets/huggingface_logo-noborder.svg"

with gr.Blocks(title="HF + Gradio Sentiment Review Console") as demo:
    gr.Markdown("# Hugging Face + Gradio: Sentiment Review Console")
    gr.Markdown(
        "This demo uses a Gradio Blocks layout (rows/columns), an image, text inputs, and a slider, "
        "and integrates a Hugging Face `transformers` pipeline."
    )

    with gr.Row():
        # LEFT COLUMN: image + controls
        with gr.Column(scale=1):
            gr.Image(
                value=HEADER_IMAGE_URL,
                label="Header Image (loaded from URL)",
                show_label=True,
                height=120
            )

            threshold = gr.Slider(
                minimum=0.50,
                maximum=0.99,
                value=0.80,
                step=0.01,
                label="Confidence Threshold"
            )

            examples = gr.Dropdown(
                choices=[
                    "I loved this product. It works perfectly and exceeded expectations.",
                    "Terrible experience. It broke after one day and support was unhelpful.",
                    "It's okay. Not great, not awful, but the price is a bit high."
                ],
                value="I loved this product. It works perfectly and exceeded expectations.",
                label="Quick Examples"
            )
            load_example_btn = gr.Button("Load Example into Textbox")
            clear_btn = gr.Button("Clear")

        # RIGHT COLUMN: main input + outputs
        with gr.Column(scale=2):
            review = gr.Textbox(
                label="Paste a review (or type your own)",
                lines=7,
                placeholder="Type a short review here..."
            )

            summary_out = gr.Textbox(label="Model Summary", lines=1)
            probs_out = gr.JSON(label="Class Probabilities")
            verdict_out = gr.Textbox(label="Threshold Verdict", lines=1)

            run_btn = gr.Button("Analyze Sentiment", variant="primary")

    # Events / wiring (must be inside Blocks)
    load_example_btn.click(fn=lambda x: x, inputs=examples, outputs=review)

    run_btn.click(
        fn=analyze_review,
        inputs=[review, threshold],
        outputs=[summary_out, probs_out, verdict_out]
    )

    clear_btn.click(
        fn=lambda: ("", "", {"NEGATIVE": 0.0, "POSITIVE": 0.0}, ""),
        inputs=[],
        outputs=[review, summary_out, probs_out, verdict_out]
    )

# Launch (prevent_thread_lock=True so the cell doesn't run indefinitely in Colab)
demo.launch(share=True, debug=False, prevent_thread_lock=True)


/usr/local/lib/python3.12/dist-packages/gradio/helpers.py:1141: UserWarning: Watermarking for SVG images is currently not supported. No watermark will be applied.
  warnings.warn(message)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5e7ebfa3887eb4206e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
